### Practical work : Linear Regression

In [ ]:
import pandas as pd
import numpy as np
import scipy as sp
import statsmodels.api as sm
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
import warnings
warnings.filterwarnings('ignore')

#### 1 - Presentation of the Boston Housing dataset

This data set concerns the task of predicting housing values in areas of Boston. The used variables and their meanings are as follows:
1. CRIM per capita crime rate by town
2. ZN proportion of residential land zoned for lots over 25,000 sq.ft.
3. INDUS proportion of non-retail business acres per town
4. CHAS Charles River dummy variable (= 1 if tract bounds river; 0 otherwise)
5. NOX nitric oxides concentration (parts per 10 million)
6. RM average number of rooms per dwelling
7. AGE proportion of owner-occupied units built prior to 1940
8. DIS weighted distances to five Boston employment centres
9. RAD index of accessibility to radial highways
10. TAX full-value property-tax rate per \$10,000
11. PTRATIO pupil-teacher ratio by town
12. LSTAT % lower status of the population
13. MEDV Median value of owner-occupied homes in $1000's

Source: UCI machine learning repository. (http://www.ics.uci.edu/~mlearn/MLSummary.html).
Characteristics: 506 cases; 12 continuous variables
Download : housing.tar.gz (11883 bytes)

In [ ]:
# rendering ++
plt.style.use('seaborn')

In [ ]:
data = pd.read_csv("BostonHousing.csv")
data.head()

In [ ]:
# Use the describe method to have basic stats about the data
data.describe()

#### 2 - Plots to visualize data and observe correlations

In [ ]:
plt.hist(data["medv"], bins=20)
plt.xlabel('price ($1000s)')
plt.ylabel('count');

In [ ]:
feature_names=data.keys()[:-1]
for u in feature_names:
    fig = data.plot(x=u, y="medv", style="o")
    fig.set_ylabel("medv", fontsize=15)
    fig.set_xlabel(u, fontsize=15)

#### 3 - Simple linear regression with 1 feature

We use the "StatsModels" Library.

In [ ]:
X = data["lstat"]
X = sm.add_constant(X)  # add the intercept term => necessary with statsmodels regression
y = data["medv"]

In [ ]:
ols = sm.OLS(y, X).fit()
ols.summary()

##### Comments

* p-values for the Student test on the coefficients are negligible: the coefficients cannot be considered to be zero.
We recall that to test the nullity of the coefficients, $H_0$ corresponds to $\beta_i=0$. Therefore, a very low $p$-value leads to reject $H_0$.

* Coefficient of determination : 
$R^{2}=1-\frac{\sum_{i=1}^{n}\left(y_{i}-\hat{y}_{i}\right)^{2}}{\sum_{i=1}^{n}\left(y_{i}-\overline{y}\right)^{2}}$

In [ ]:
# Drawing the regression line on top of the scatterplot
fig = data.plot(x="lstat", y="medv", style="o")
fig.set_ylabel("medv")
ypred = ols.predict(X)
fig.plot(data['lstat'], ypred, 'r', linewidth=2.5)
fig.set_title('R^2 = {:.2f}'.format(ols.rsquared))

In [ ]:
residuals = y - ypred
fig = sm.qqplot(residuals, dist="norm", line="r")

For a Normal distributions of the error terms in the regression model, the residual would be distributed close to the red line. Therefore, here it's difficult to validate the asssumption of normal residuals.

#### 4 - Linear regression with 2 features, lstat and age

In [ ]:
X = data[["age", "lstat"]]
X = sm.add_constant(X)  # add the intercept term => necessary with statsmodel regression
y = data["medv"]
ols2 = sm.OLS(y, X).fit()
ols2.summary()

##### Comments

* AIC=3281, while it was 3287 with only 2 parameters (1 feature): the model with 3 parameters (2 features) is better.

* Again, $p$-values are really low and the parameters cannot be considered to be zero

#### 5 - Linear regression with all features and model selection


In [ ]:
# the features are all columns but the last one
features = data.keys()[:-1]
X = data[features]
X = sm.add_constant(X)  # add the intercept term => necessary to
y = data["medv"]
ols3 = sm.OLS(y, X).fit()
ols3.summary()

##### Comments

* AIC=3036, the model with all features is better.

* $p$-values for the parameters associated to 'indus' and 'age' are high, which suggests that the coefficients should be set to 0. However, we usually do it sequentially, first by considering the model without indus, checking again the $p$-values and iterate if necessary.

In [ ]:
# we use all features but 'indus'
features = features.drop('indus')
X = data[features]
X = sm.add_constant(X)  # add the intercept term => necessary to
y = data["medv"]
ols3 = sm.OLS(y, X).fit()
ypred=ols3.predict(X)
ols3.summary()

In [ ]:
residuals = y - ypred
fig = sm.qqplot(residuals, dist="norm", line="r")

Slightly better in terms of residuals normality.

##### Comments

* AIC=3034, the model with all features but indus is better than the previous one.

* $p$-values for the parameters 'age' is still high, which suggests that the coefficients should be set to 0.

In [ ]:
# we use all features but 'indus'
features = features.drop('age')
X = data[features]
X = sm.add_constant(X)  # add the intercept term
y = data["medv"]
ols4 = sm.OLS(y, X).fit()
ols4.summary()

##### Comments

* AIC=3032, the model with all features but indus and age is better than the previous ones.

* $p$-values for all parameters are negligible: it does not seem possible ro reduce the model furthermore a priori.

#### 6 - Prediction Test

A classical way to evaluate a prediction function in statistical learning, is to spare one part of the dataset for testing purpose (classically 20%). The dataset is thus split into a "training set" and a "test set". The rule to never forget is that in such configuration, the test set should not be used at all in the training process (notably for the selection of hyperparameters): it is used only to assess the prediction capacity of a fully trained model.

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
ols5 = sm.OLS(y_train, X_train).fit()
ols5.summary()

In [ ]:
predicted = ols5.predict(X_test)
expected = y_test

plt.scatter(expected, predicted)
plt.plot([0, 50], [0, 50], '--k')
plt.axis('tight')
plt.xlabel('True price ($1000s)')
plt.ylabel('Predicted price ($1000s)')
print("RMS: {:.2f}".format(np.sqrt(np.mean((predicted - expected) ** 2))))

#### 7 - Regularization

We explore the elastic-net method, which is a combination of Lasso and Ridge-Penalty:

$\hat{\beta} \equiv \underset{\beta}{\operatorname{argmin}}\frac{1}{2N}\|y-X \beta\|^{2}+\alpha\left((1-\lambda_1)\frac{\displaystyle|\beta\|^{2}}{2}+\lambda_{1}\|\beta\|_{1}\right)$

Be careful: we need to standardize the input data !

We use scikitlearn for this purpose.

In [ ]:
features = data.keys()[:-1]
X = data[features]
y = data["medv"]

#we standardize X
from sklearn import preprocessing
scaler_x= preprocessing.StandardScaler().fit(X)
X_std=scaler_x.transform(X)

y_centered=y-np.mean(y)

##### 7.a - The Lasso

In [ ]:
from sklearn.linear_model import ElasticNet

###### Lasso Regression Path

In [ ]:
from sklearn.linear_model import ElasticNet
als=np.logspace(-3, 1.5, 200)
coefs=[]
for al in als:
    reg = ElasticNet(alpha=al,l1_ratio=1)
    reg.fit(X_std, y_centered)
    coefs.append(reg.coef_)
 

In [ ]:
# A list of different colors to draw our coefficients
colormap = plt.cm.gist_ncar 
colorst = [colormap(i) for i in np.linspace(0, 0.95,len(features))]

In [ ]:
for i in range(len(features)):
    plt.plot(np.log10(als), [x[i] for x in coefs], label=features[i], color=colorst[i])
plt.xlabel(r'$\ln(\alpha)$')
plt.ylabel('Coefficients')
plt.title('Lasso Regression Path')
plt.legend(bbox_to_anchor=(1.04,0.5), loc="center left", borderaxespad=0)
plt.axis('tight')
#ymin, ymax = plt.ylim()
#plt.axis([0,0.2,-1,1],'tight')

###### Estimation of the Generalization Error by Cross Validation

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn import metrics
features=list(data)[0:-1]
X = data[features]
scaler_x= preprocessing.StandardScaler().fit(X)
X_std=scaler_x.transform(X)

ols = LinearRegression(fit_intercept=False)
score_ols = cross_val_score(ols, X_std, y_centered, cv=5, 
                            scoring=('neg_mean_squared_error'))
print("Neg MSE for the full model:",np.mean(score_ols))

In [ ]:
from sklearn.model_selection import GridSearchCV

clf = GridSearchCV(ElasticNet(l1_ratio=1, fit_intercept=False), 
                   param_grid={'alpha': np.logspace(-1.5, -0.5, 500)}, 
                   cv=5, 
                   scoring='neg_mean_squared_error')
clf.fit(X_std, y_centered)

In [ ]:
plt.plot(np.log10(clf.param_grid['alpha']), 
         clf.cv_results_['mean_test_score'])
plt.xlabel(r'$\ln(\alpha)$')
plt.ylabel('Neg MSE')
plt.title('Evolution of the Cross Validation Error with the penalty '\
          'coefficient for the Lasso estimator')


In [ ]:
best_alpha = clf.best_params_['alpha']
print("best alpha:", best_alpha,
      "\n","best MSE:", clf.best_score_)

In [ ]:
reg = ElasticNet(alpha=best_alpha, l1_ratio=1)
reg.fit(X_std, y)
for i in range(len(features)):
    print("coeff {} : {:.2f}".format(features[i], reg.coef_[i]))

Only one coefficient is zero: that of 'age'.
We perform an Ordinary Linear Regression (without the penalty term) and by setting to 0 the coefficient $\beta_{age}$: the idea is to check whether we perform better by removing the bias due to the penalty term.

In [ ]:
features=list(data)[0:-1]
features.remove("age")
X = data[features]
#we standardize X
scaler_x= preprocessing.StandardScaler().fit(X)
X_std=scaler_x.transform(X)

ols = LinearRegression(fit_intercept=False)
score_ols = cross_val_score(ols, X_std, y_centered, cv=5, 
                            scoring=('neg_mean_squared_error'))

print("MSE for the model with all features but age and indus:",np.mean(score_ols))


It is indeed slightly better. Since with the original Ordinary Least-Square regression, the model without "indus" and "age" was pretty good, we also compute the Cross Validation Error for this model.

In [ ]:
features=list(data)[0:-1]
features.remove("age")
features.remove("indus") #in addition to "age", we also remove "indus"
X = data[features]
#we standardize X
scaler_x= preprocessing.StandardScaler().fit(X)
X_std=scaler_x.transform(X)

ols = LinearRegression(fit_intercept=False)
score_ols = cross_val_score(ols, X_std, y_centered, cv=5, scoring=('neg_mean_squared_error'))
print("MSE for the model with all features but age and indus:",np.mean(score_ols))



It is indeed better. The strategy set in place with the lasso failed to find the best model.

In [ ]:
# Idem for the ridge regression
als=np.logspace(-2, 5, 100)
coefs=[]
for al in als:
    reg = ElasticNet(alpha=al,l1_ratio=0,fit_intercept=False) 
    reg.fit(X_std, y)
    coefs.append(reg.coef_)
for i in range(len(features)):
    plt.plot(np.log10(als), [x[i] for x in coefs], label=features[i], color=colorst[i])
plt.xlabel(r'$\ln(\alpha)$')
plt.ylabel('Coefficients')
plt.title('Ridge Regression Path')
l3 = plt.legend(bbox_to_anchor=(1.04,0.5), loc="center left", borderaxespad=0)
plt.axis('tight')

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn import metrics
features=list(data)[0:-1]
X = data[features]
scaler_x= preprocessing.StandardScaler().fit(X)
X_std=scaler_x.transform(X)
y_centered=y-np.mean(y)

clf = GridSearchCV(ElasticNet(l1_ratio=0, fit_intercept=False), 
                   param_grid={'alpha': np.logspace(-2, 2, 200)}, 
                   cv=5, 
                   scoring='neg_mean_squared_error')
clf.fit(X_std, y_centered)

In [ ]:
plt.plot(np.log10(clf.param_grid['alpha']), clf.cv_results_['mean_test_score'])
plt.xlabel(r'$\ln(\alpha)$')
plt.ylabel('Neg MSE')
plt.title('Evolution of the Cross Validation Error with the penalty '\
          'coefficient for the ridge estimator')


In [ ]:
best_alpha=clf.best_params_['alpha']
print("best alpha:", best_alpha,"\n","best MSE:",clf.best_score_)

The ridge regression provides the best configuration between ordinary least-square, Lasso and ridge, at least in terms of Cross Validation prediction error, which can be seen as an approximation of the generalization error.

##### 8 - Back to Data Visualization

One reason for the relatively disappointing performance of Lasso in this case is the correlations between features. Ridge is known to perform better in this situation. 

We can visualize the correlations of the data with several tools. The firs one is the scattermatrix: it shows the pairwise scatter plots for all the covariates (as well as the target for the graph below).

We can use directly a function from the pandas module (note that the matrix of plots is symmetrical; on the diagonal, we show the histogram of the variables. (Be patient, it takes a litlle time to proceed....)

In [ ]:
import seaborn as sns

In [ ]:
fig, ax = plt.subplots(figsize=(15, 12))
sns.heatmap(data.corr(), cmap='seismic', annot=True, ax=ax)

In [ ]:
grr = pd.plotting.scatter_matrix(data, c='darkorchid', 
                                 figsize=(15, 15), marker='o',
                                 hist_kwds={'bins': 20, 'color' : 'darkmagenta'}, 
                                 s=10, alpha=.6)


Another possibility would be to use the seaborn package, which provides nice data visualization tools

In [ ]:
import seaborn as sns
sns.pairplot(data)

Finally, the correlation matrix can help us summarize all this information. 
We use great customized tool for this purpose !

Strong correlations between features appear strikingly.

In [ ]:
#### Thanks to: https://github.com/drazenz/heatmap/blob/master/heatmap.py
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np


def heatmap(x, y, **kwargs):
    if 'color' in kwargs:
        color = kwargs['color']
    else:
        color = [1]*len(x)

    if 'palette' in kwargs:
        palette = kwargs['palette']
        n_colors = len(palette)
    else:
        n_colors = 256 # Use 256 colors for the diverging color palette
        palette = sns.color_palette("Blues", n_colors) 

    if 'color_range' in kwargs:
        color_min, color_max = kwargs['color_range']
    else:
        color_min, color_max = min(color), max(color) # Range of values that will be mapped to the palette, i.e. min and max possible correlation

    def value_to_color(val):
        if color_min == color_max:
            return palette[-1]
        else:
            val_position = float((val - color_min)) / (color_max - color_min) # position of value in the input range, relative to the length of the input range
            val_position = min(max(val_position, 0), 1) # bound the position betwen 0 and 1
            ind = int(val_position * (n_colors - 1)) # target index in the color palette
            return palette[ind]

    if 'size' in kwargs:
        size = kwargs['size']
    else:
        size = [1]*len(x)

    if 'size_range' in kwargs:
        size_min, size_max = kwargs['size_range'][0], kwargs['size_range'][1]
    else:
        size_min, size_max = min(size), max(size)

    size_scale = kwargs.get('size_scale', 500)

    def value_to_size(val):
        if size_min == size_max:
            return 1 * size_scale
        else:
            val_position = (val - size_min) * 0.99 / (size_max - size_min) + 0.01 # position of value in the input range, relative to the length of the input range
            val_position = min(max(val_position, 0), 1) # bound the position betwen 0 and 1
            return val_position * size_scale
    if 'x_order' in kwargs: 
        x_names = [t for t in kwargs['x_order']]
    else:
        x_names = [t for t in sorted(set([v for v in x]))]
    x_to_num = {p[1]:p[0] for p in enumerate(x_names)}

    if 'y_order' in kwargs: 
        y_names = [t for t in kwargs['y_order']]
    else:
        y_names = [t for t in sorted(set([v for v in y]))]
    y_to_num = {p[1]:p[0] for p in enumerate(y_names)}

    plot_grid = plt.GridSpec(1, 15, hspace=0.2, wspace=0.1) # Setup a 1x10 grid
    ax = plt.subplot(plot_grid[:,:-1]) # Use the left 14/15ths of the grid for the main plot

    marker = kwargs.get('marker', 's')

    kwargs_pass_on = {k:v for k,v in kwargs.items() if k not in [
         'color', 'palette', 'color_range', 'size', 'size_range', 'size_scale', 'marker', 'x_order', 'y_order'
    ]}

    ax.scatter(
        x=[x_to_num[v] for v in x],
        y=[y_to_num[v] for v in y],
        marker=marker,
        s=[value_to_size(v) for v in size], 
        c=[value_to_color(v) for v in color],
        **kwargs_pass_on
    )
    ax.set_xticks([v for k,v in x_to_num.items()])
    ax.set_xticklabels([k for k in x_to_num], rotation=45, horizontalalignment='right')
    ax.set_yticks([v for k,v in y_to_num.items()])
    ax.set_yticklabels([k for k in y_to_num])

    ax.grid(False, 'major')
    ax.grid(True, 'minor')
    ax.set_xticks([t + 0.5 for t in ax.get_xticks()], minor=True)
    ax.set_yticks([t + 0.5 for t in ax.get_yticks()], minor=True)

    ax.set_xlim([-0.5, max([v for v in x_to_num.values()]) + 0.5])
    ax.set_ylim([-0.5, max([v for v in y_to_num.values()]) + 0.5])
    ax.set_facecolor('#F1F1F1')

    # Add color legend on the right side of the plot
    if color_min < color_max:
        ax = plt.subplot(plot_grid[:,-1]) # Use the rightmost column of the plot

        col_x = [0]*len(palette) # Fixed x coordinate for the bars
        bar_y=np.linspace(color_min, color_max, n_colors) # y coordinates for each of the n_colors bars

        bar_height = bar_y[1] - bar_y[0]
        ax.barh(
            y=bar_y,
            width=[5]*len(palette), # Make bars 5 units wide
            left=col_x, # Make bars start at 0
            height=bar_height,
            color=palette,
            linewidth=0
        )
        ax.set_xlim(1, 2) # Bars are going from 0 to 5, so lets crop the plot somewhere in the middle
        ax.grid(False) # Hide grid
        ax.set_facecolor('white') # Make background white
        ax.set_xticks([]) # Remove horizontal ticks
        ax.set_yticks(np.linspace(min(bar_y), max(bar_y), 3)) # Show vertical ticks for min, middle and max
        ax.yaxis.tick_right() # Show vertical ticks on the right 


def corrplot(data, size_scale=280, marker='s'):
    corr = pd.melt(data.reset_index(), id_vars='index')
    corr.columns = ['x', 'y', 'value']
    heatmap(
        corr['x'], corr['y'],
        color=corr['value'], color_range=[-1, 1],
        palette=sns.diverging_palette(20, 220, n=256),
        size=corr['value'].abs(), size_range=[0,1],
        marker=marker,
        x_order=data.columns,
        y_order=data.columns[::-1],
        size_scale=size_scale
    )
    
plt.figure(figsize=(8,8))
corrplot(data.corr())

That's all for today ...

Or quite ...

A last exercise for you:

We selected the hyper-parameters for the Lasso and Ridge estimators respectively. The interest of the elastic net is that it sometimes can combine the advantages of both. 

Define a 2D grid search for both $\alpha$ and $\lambda_1$ and select the 2 best hyperparameters by cross-validation.




In [ ]:
clf = GridSearchCV(ElasticNet(fit_intercept=False), 
                  param_grid={'alpha': np.logspace(-1.5, 0.5, 50), 
                             'l1_ratio':  np.logspace(-1.5, 0.5, 50)})
clf.fit(X_std, y_centered)